# Introduction

This notebook will cover customer churn prediction, 
trying to predict customers with a higher probability of continuing with their account or not.

I'll try to address the problem using this framework:

**framework:**

* Problem definition
* Data exploration
* evaluation
* Features
* Modelling
* Experimentationation

## Importing Tools

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# for plots to appear inside notebook
%matplotlib inline

## Explore Data

#### load data

In [4]:
# import both train data 
customer_train_data = pd.read_csv("data/playground-series-s4e/train.csv")
customer_train_data.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [ ]:
import test d